In [1]:
1 + 2

3

In [4]:
def fich(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fich(n-1) + fich(n-2)
    
fich(12)

233

In [1]:
pow(100,2)

10000

In [2]:
max(1, -2, 3, 5)

5

In [3]:
max(min(1, -2), min(pow(3,5), -4))

-2

In [5]:
from math import sqrt
sqrt(256)

16.0

In [ ]:
from operator import add, sub, mul
add(14, 28)
sub(100,)